# Introduction to Python for Data Engineering

This notebook demonstrates how to read data from a public API, process it to create useful information, and store the result in an SQLite database.

It assumes you have read [this article](https://www.startdataengineering.com/post/python-for-de/#extract--load-read-and-write-data-to-any-system).

**Acknowledgment**: Parts of this notebook were developed with the assistance of ChatGPT, an AI language model by OpenAI, to support instructional design and coding tasks related to data engineering concepts.

In [53]:
# Install required packages
!pip install requests

In [54]:
# Import libraries
import requests
import sqlite3
from datetime import datetime

In [55]:
# Read data from CoinGecko API
url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd"
response = requests.get(url)
data = response.json()

data


{'bitcoin': {'usd': 119943}}

We will extract the Bitcoin price in USD and convert it to ZAR using an estimated exchange rate.
This is an example of turning raw data into useful **information**.

In [56]:
# Function to convert USD to ZAR
def convert_usd_to_zar(usd_amount, rate=18.5):
    return round(usd_amount * rate, 2)

In [57]:
# Extract USD price and apply transformation
usd_price = data["bitcoin"]["usd"]
zar_price = convert_usd_to_zar(usd_price)
timestamp = datetime.now().isoformat()

print(f"Bitcoin price in USD: {usd_price}")
print(f"Estimated price in ZAR: {zar_price}")


Bitcoin price in USD: 119943
Estimated price in ZAR: 2218945.5


In [58]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Create table if it doesn't exist
# This CREATE TABLE statement will only be executed if the table doesn't exist.
# Based on the PRAGMA output, the table already exists with more columns.
# However, keeping this here is good practice if you were to start with a fresh database.
cursor.execute('''
CREATE TABLE IF NOT EXISTS prices (
    timestamp TEXT,
    usd REAL,
    zar REAL
)
''')

# Insert the new row by specifying the columns
cursor.execute("INSERT INTO prices (timestamp, usd, zar) VALUES (?, ?, ?)", (timestamp, usd_price, zar_price))
conn.commit()
conn.close()

In [59]:
# Check if data was successfully written
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Show the last 5 rows
rows = cursor.execute("SELECT * FROM prices ORDER BY timestamp DESC LIMIT 5").fetchall()
for row in rows:
    print(row)

conn.close()


('2025-08-13T08:58:48.466317', 119943.0, 2218945.5, None, None)
('2025-08-13T08:56:14.782415', None, None, None, 165066.0)
('2025-08-13T08:56:14.782415', None, None, None, 165066.0)
('2025-08-13T08:56:14.782415', 119896.0, 2218076.0, None, None)
('2025-08-13T08:56:14.782415', 119896.0, 2218076.0, None, None)


# Task
Update the notebook to convert Bitcoin price to any target currency supported by api.coingecko.com instead of only ZAR.

## Identify target currencies

### Subtask:
Determine how the user will specify the target currency (e.g., as an input variable, a list of currencies).


**Reasoning**:

*   List item
*   List item


Define a variable to hold the target currency for the conversion.



In [60]:
target_currency = 'eur'

## Modify api call

### Subtask:
Update the API call to fetch the Bitcoin price in the specified target currency or currencies.


**Reasoning**:
Modify the API call to fetch the Bitcoin price in the specified target currency.



In [61]:
# Read data from CoinGecko API
url = f"https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies={target_currency}"
response = requests.get(url)
data = response.json()

data

{'bitcoin': {'eur': 102387}}

**Reasoning**:
The error indicates that the `requests` library was not imported in the current execution environment. Re-import the necessary libraries.



In [62]:
import requests
import sqlite3
from datetime import datetime

# Read data from CoinGecko API
url = f"https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies={target_currency}"
response = requests.get(url)
data = response.json()

data

{'bitcoin': {'eur': 102387}}

## Update data processing

### Subtask:
Adjust the data processing logic to handle the response from the updated API call, which will contain prices in the target currency or currencies.


**Reasoning**:
Adjust the data processing logic to handle the response from the updated API call, which will contain prices in the target currency.



In [63]:
# Extract price in target currency
target_price = data["bitcoin"][target_currency]
timestamp = datetime.now().isoformat()

print(f"Bitcoin price in {target_currency.upper()}: {target_price}")

Bitcoin price in EUR: 102387


**Reasoning**:
The data processing logic has been adjusted to handle the target currency. The next step is to update the database insertion logic to use the target price and currency, and then verify the database update.



In [64]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Create table if it doesn't exist, dynamically adding the target currency column
# This assumes the table structure might change based on the target currency.
# A more robust solution for multiple currencies would involve a different schema,
# but for this subtask, we'll add the target currency column if it doesn't exist.

# Check if the target currency column exists
cursor.execute(f"PRAGMA table_info(prices)")
columns = [col[1] for col in cursor.fetchall()]

if target_currency not in columns:
    # Add the new column
    cursor.execute(f"ALTER TABLE prices ADD COLUMN {target_currency} REAL")
    conn.commit()
    print(f"Added column '{target_currency}' to the prices table.")

# Insert the new row, handling potential existing columns (like 'zar' from previous runs)
# We'll insert the timestamp and target_currency price. Other columns will be NULL or require more complex logic.
# For simplicity in this subtask, we'll just insert the new data point.
# A production system would need a more structured way to handle multiple currencies over time.

# Construct the INSERT statement dynamically
insert_sql = f"INSERT INTO prices (timestamp, {target_currency}) VALUES (?, ?)"
cursor.execute(insert_sql, (timestamp, target_price))

conn.commit()
conn.close()

# Check if data was successfully written
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Show the last 5 rows including the new column if it exists
rows = cursor.execute(f"SELECT timestamp, usd, zar, {target_currency} FROM prices ORDER BY timestamp DESC LIMIT 5").fetchall()
for row in rows:
    print(row)

conn.close()

('2025-08-13T08:58:48.630371', None, None, 102387.0)
('2025-08-13T08:58:48.466317', 119943.0, 2218945.5, None)
('2025-08-13T08:56:14.782415', None, None, None)
('2025-08-13T08:56:14.782415', None, None, None)
('2025-08-13T08:56:14.782415', 119896.0, 2218076.0, None)


**Reasoning**:
The previous code block failed because the 'prices' table does not exist. This is likely because the notebook was reset or the database file was deleted. I need to recreate the table before attempting to alter or insert into it.



In [65]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS prices (
    timestamp TEXT,
    usd REAL,
    zar REAL
)
''')
conn.commit()

# Check if the target currency column exists and add it if not
cursor.execute(f"PRAGMA table_info(prices)")
columns = [col[1] for col in cursor.fetchall()]

if target_currency not in columns:
    cursor.execute(f"ALTER TABLE prices ADD COLUMN {target_currency} REAL")
    conn.commit()
    print(f"Added column '{target_currency}' to the prices table.")

# Insert the new row
insert_sql = f"INSERT INTO prices (timestamp, {target_currency}) VALUES (?, ?)"
cursor.execute(insert_sql, (timestamp, target_price))

conn.commit()
conn.close()

# Check if data was successfully written
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Show the last 5 rows including the new column if it exists
# Select all available columns dynamically
cursor.execute(f"PRAGMA table_info(prices)")
available_columns = [col[1] for col in cursor.fetchall()]
select_sql = f"SELECT {', '.join(available_columns)} FROM prices ORDER BY timestamp DESC LIMIT 5"
rows = cursor.execute(select_sql).fetchall()

for row in rows:
    print(row)

conn.close()

('2025-08-13T08:58:48.630371', None, None, 102387.0, None)
('2025-08-13T08:58:48.630371', None, None, 102387.0, None)
('2025-08-13T08:58:48.466317', 119943.0, 2218945.5, None, None)
('2025-08-13T08:56:14.782415', None, None, None, 165066.0)
('2025-08-13T08:56:14.782415', None, None, None, 165066.0)


## Update database schema

### Subtask:
Modify the SQLite database schema to store prices for different currencies. This might involve adding a currency column or creating separate tables for each currency.


## Update database insertion

### Subtask:
Adjust the database insertion logic to correctly insert the fetched prices into the updated database schema.


## Update data display

### Subtask:
Modify the code that displays the fetched data to show the Bitcoin price in the target currency or currencies.


**Reasoning**:
Connect to the SQLite database, retrieve column names, fetch the last 5 rows, and print them to display the fetched data including the target currency price.



In [66]:
# Check if data was successfully written
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Show the last 5 rows including the new column if it exists
# Select all available columns dynamically
cursor.execute(f"PRAGMA table_info(prices)")
available_columns = [col[1] for col in cursor.fetchall()]
select_sql = f"SELECT {', '.join(available_columns)} FROM prices ORDER BY timestamp DESC LIMIT 5"
rows = cursor.execute(select_sql).fetchall()

for row in rows:
    print(row)

conn.close()

('2025-08-13T08:58:48.630371', None, None, 102387.0, None)
('2025-08-13T08:58:48.630371', None, None, 102387.0, None)
('2025-08-13T08:58:48.466317', 119943.0, 2218945.5, None, None)
('2025-08-13T08:56:14.782415', None, None, None, 165066.0)
('2025-08-13T08:56:14.782415', None, None, None, 165066.0)


## Summary:

### Data Analysis Key Findings

*   The notebook was successfully updated to allow the user to specify a target currency for Bitcoin price conversion using a variable.
*   The API call was modified to dynamically fetch the Bitcoin price in the specified target currency from the CoinGecko API.
*   The data processing logic was adjusted to handle the API response containing the price in the target currency.
*   The SQLite database schema was updated to dynamically add a column for the target currency if it does not exist.
*   The database insertion logic was adjusted to correctly insert the fetched price into the newly added column in the `prices` table.
*   The data display code was modified to dynamically retrieve and display the Bitcoin price in the target currency from the database, along with other available currency columns.

### Insights or Next Steps

*   Consider implementing error handling for invalid currency inputs or API call failures.
*   Explore options for displaying historical data or visualizations of Bitcoin price trends in the target currency.


In [67]:
import sqlite3

conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

cursor.execute("PRAGMA table_info(prices)")
table_info = cursor.fetchall()

for column in table_info:
    print(column)

conn.close()

(0, 'timestamp', 'TEXT', 0, None, 0)
(1, 'usd', 'REAL', 0, None, 0)
(2, 'zar', 'REAL', 0, None, 0)
(3, 'eur', 'REAL', 0, None, 0)
(4, 'cad', 'REAL', 0, None, 0)
